In [ ]:
import netsquid as ns
# Set qstate formalism to use density matrices.
ns.set_qstate_formalism(ns.QFormalism.DM)

# Tutorial 3: Quantum network example

## Network setup

In [ ]:
from netsquid.nodes import Node
from netsquid.components import Port, QuantumChannel
from netsquid.nodes import DirectConnection
from netsquid.components.models import FixedDelayModel, DepolarNoiseModel

alice_node = Node("Alice")
bob_node   = Node("Bob")

delay_model = FixedDelayModel(delay=10.)
qerror_model = DepolarNoiseModel(depolar_rate=0.2, time_independent=True)

channel_a_to_b = QuantumChannel("Channel Alice to Bob",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": qerror_model}
                                  )

channel_b_to_a = QuantumChannel("Channel Bob to Alice",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": qerror_model}
                                  )


connection = DirectConnection(name="connection",
                              channel_AtoB=channel_a_to_b,
                              channel_BtoA=channel_b_to_a)

alice_port = Port("Alice_port", alice_node)
bob_port   = Port("Bob_port",   bob_node)

alice_node.connect_to(remote_node=bob_node, connection=connection,
                     local_port_name="Alice_port", remote_port_name="Bob_port")

### Component diagram
![title](img/network_components_diagram.drawio.png)

## Protocols

In [ ]:
from netsquid.protocols import NodeProtocol

class AliceProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Alice_port"]

        q1, q2 = ns.qubits.create_qubits(2)
        
        ns.qubits.assign_qstate([q1, q2], ns.b00)
        
        port.tx_output(q2)
        print(f"{ns.sim_time()} ns Alice send q2")

        # Wait 30 ns
        yield self.await_timer(30)

        m, _ = ns.qubits.measure(q1)

        print(f"{ns.sim_time()} ns Alice measures q1: {m}")



class BobProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Bob_port"]

        yield self.await_port_input(port)
        
        q2 = port.rx_input().items[0]

        dm = ns.qubits.reduced_dm(q2.qstate.qubits)
        print(f"{ns.sim_time()} ns Bob receives q2, DM=\n{dm}")

        m, _ = ns.qubits.measure(q2)

        print(f"{ns.sim_time()} ns Bob measures q2: {m}")

alice_protocol = AliceProtocol(node=alice_node)
bob_protocol = BobProtocol(node=bob_node)

## Timeline diagram
![title](img/tutorial3_timeline.drawio.png)

## Run simulation

In [ ]:
# Start all protocols
alice_protocol.start()
bob_protocol.start()

sim_stats = ns.sim_run()

print(sim_stats)

# Reset simulation
alice_protocol.stop()
bob_protocol.stop()

ns.sim_reset()

# Quantum error models

In [ ]:
from netsquid.components.models import DepolarNoiseModel, DephaseNoiseModel, T1T2NoiseModel, FibreLossModel

# Apply depolarising noise to qubits with 20% chance
depolar_noise_model = DepolarNoiseModel(depolar_rate=0.2, time_independent=True)

# Apply dephasing noise to qubits with 20% chance
dephase_noise_model = DephaseNoiseModel(dephase_rate=0.2, time_independent=True)
# If time_independent is True, both noise models will have a fixed probability of noise, with p=rate
# If time_independent is False. The noise will be modelded p = e^{-T/rate}, where rate is in Hz. 

# Phenomenological noise model. T1 time dicates amplitude dampening component, T2 dictates dephasing component
t1_t2_noise_model = T1T2NoiseModel(T1=100, T2=20)


# Exponential photon loss model, probability that qubit object does not arrive at destination
# 20% Fixed loss probability and 0.25dB/km 
fibre_loss_model = FibreLossModel(p_loss_init=0.2, p_loss_length=0.25)

# Transmission success probability = 0.8 * 10^(-0.25dB/km * 15km / 10dB) ~ 0.45
length = 15
channel_a_to_b = QuantumChannel("Channel Alice to Bob",
                                models={"quantum_loss_model": fibre_loss_model},
                                length=length
                                )


Next: [Exercise](../exercises/exercise1.ipynb)